# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [ ]:
# 1) import any other libraries you might need
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# 2) instantiate a Spark session 
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

In [ ]:
# 3) read in the data set located at the path "data/sparkify_log_small.json"
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [ ]:
user_log.printSchema()

# Write code to answer the quiz questions 

# Question 1

Which page did user id "" (empty string) NOT visit?

In [ ]:
# filter for users with blank user id
blank_pages = user_log.filter(user_log.userId == '').select('page').alias('blank_pages').dropDuplicates()

In [ ]:
# get a list of possible pages that could be visited
all_pages = user_log.select('page').dropDuplicates()

In [ ]:
# find values in all_pages that are not in blank_pages
# these are the pages that the blank user did not go to
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

# Question 2
What type of user does the empty string user id most likely refer to?

In [ ]:
# TODO: use this space to explore the behavior of the user with an empty string


# Question 3

How many female users do we have in the data set?

In [ ]:
# TODO: write your code to answer question 3
user_log.filter(user_log.gender == 'F').select('userId', 'gender').dropDuplicates().count()

# Question 4

How many songs were played from the most played artist?

In [ ]:
# TODO: write your code to answer question 4
user_log.filter(user_log.page == 'NextSong') \
    .select('Artist').groupBy('Artist').agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount').sort(desc('Artistcount')).show(1)

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [ ]:
# TODO: write your code to answer question 5
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

In [ ]:
user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

In [ ]:
cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function('page')) \
    .withColumn('period', Fsum('homevisit').over(user_window))

In [ ]:
cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()